In [1]:
print("This part is the test of SBert model")

from sentence_transformers import SentenceTransformer, util
sentences_test = ["I'm happy", "I'm full of happiness"]

print("Loading SBert....")
SBert_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')


print("Computing Embeddings....")
embedding_1= SBert_model.encode(sentences_test[0], convert_to_tensor=True)
embedding_2 = SBert_model.encode(sentences_test[1], convert_to_tensor=True)

print("Sentence Similarity:", util.pytorch_cos_sim(embedding_1, embedding_2))

print("Expected Out:tensor[[0.6003]]")

This part is the test of SBert model
Loading SBert....
Computing Embeddings....
Sentence Similarity: tensor([[0.6003]])
Expected Out:tensor[[0.6003]]


In [2]:
print("This part is the test of Word2Vec model")

from gensim.models import Word2Vec
import gensim
import numpy as np

print("Loading Word2Vec...")
wordvec = gensim.models.KeyedVectors.load_word2vec_format("word2vec_model.bin", binary=True)
index2word_set = set(wordvec.index_to_key)

# get average vectors
def avg_feature_vector(sentence, model, num_features, index2word_set):
    words = sentence.split()
    feature_vec = np.zeros((num_features, ), dtype='float32')
    n_words = 0
    for word in words:
        if word in index2word_set:
            n_words += 1
            feature_vec = np.add(feature_vec, model[word])
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

sentences_test = ["add support for iphone X", "where is the iphone X support"]
print("Computing sentence vectors....")
vec_1 = avg_feature_vector(sentences_test[0], model=wordvec, num_features=300, index2word_set=index2word_set)
vec_2 = avg_feature_vector(sentences_test[1], model=wordvec, num_features=300, index2word_set=index2word_set)

print("Sentence Similarity:", util.pytorch_cos_sim(vec_1, vec_2))

print("Expected Out:tensor[[0.7939]]")

This part is the test of Word2Vec model
Loading Word2Vec...
Computing sentence vectors....
Sentence Similarity: tensor([[0.7939]])
Expected Out:tensor[[0.7939]]


In [3]:
# This cell is to read data from Excel

import pandas as pd
import numpy as np

df = pd.read_excel("info_reviews/ZOOM_Informative_Reviews.xlsx","Sheet1")

dates = df["date"].tolist()
ratings = df["rating"].tolist()
sentences = df["sentence"].tolist()
print(len(dates),len(ratings),len(sentences))

reviews = []
for i in range(len(sentences)):
    reviews.append(str(ratings[i])+"&"+str(dates[i])+"&"+str(sentences[i]))


39128 39128 39128


In [4]:
# This cell is to calculate similarity with SBert model

rn_embedding = SBert_model.encode("iOS screen share support", convert_to_tensor=True)

sims = []
index = 0
l = len(sentences)

for sentence in sentences:
    embedding = SBert_model.encode(sentence,convert_to_tensor = True)
    sim = util.pytorch_cos_sim(rn_embedding, embedding)
    print('process:{0}%'.format(round(index+1)*100/l),end='\r')
    index += 1
    sims.append(float(sim[0][0]))


In [5]:
# This cell is is merge reviews and similarity in a dictionary

from collections import defaultdict
dic = defaultdict(int)

for i in range(len(sentences)):
    dic[reviews[i]] = sims[i]

In [6]:
# Get the top80

dic_order=sorted(dic.items(),key=lambda x:x[1],reverse=True)
SBert_top80s=dic_order[:80]
print(len(SBert_top80s))
print(SBert_top80s[0:3])

80
[('2&2020-04-25 01:01:54&Ok now How do you share screen on iOS ', 0.7794670462608337), ('2&2021-02-20 11:37:39&You can not screen share on a iPad ', 0.7504422068595886), ('4&2020-05-24 07:41:00&I wish you could screen share on iPad ', 0.7500161528587341)]


In [15]:
# This step is data processing for Word2Vec

import nltk
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
# 这里如果报错的话  科学上网可以解决（但是科学上网得重启jupyter 建议去anaconda prompt中输入python 然后download）

import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize, pos_tag


def clean_text(text):
    # clean datas
     # clean the new line
     text = text.replace('\n', " ")  
     # clean the url
     # text = re.sub(r"/[a-zA-Z]*[:\//\]*[A-Za-z0-9\-_]+\.+[A-Za-z0-9\.\/%&=\?\-_]+/i", "", text)
     text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', text, flags=re.MULTILINE)
     # clean the email address
     text = re.sub(r"[\w]+@[\.\w]+", "", text)
     # clean the digits
     text = re.sub(r"[0-9]", "", text)
     # clean the special charactors
     text = re.sub('[^A-Za-z0-9]+', " ", text)
     # clean the words length less than 2
     # text = ' '.join(word for word in text.split() if len(word) > 2)
     return text

# clean sentences
data = df["sentence"]
data = data.apply(lambda s: clean_text(s))
doclist = data.values.tolist() 

# remove stopwords
en_stops = set(stopwords.words('english')) 
texts = [[word for word in doc.lower().split() if word not in en_stops] for doc in doclist]

# 仅保留动词，形容词与名词
# JJ, JJR, JJS: 形容词，形容词比较级，形容词最高级
# NN, NNS, NNP, NNPS： 名词，名词复数，专有名词，专有名词复数
# VB, VBD, VBG, VBP, VBN, VBZ: 动词，动词过去式，动词现在分词，动词过去分词，动词现在式非第三人称时态，动词现在式第三人称时态
corps = set(['JJ','JJR', 'JJS',
             'NN', 'NNS', 'NNP', 'NNPS',
             'VB','VBD', 'VBG','VBP', 'VBN', 'VBZ'])

result = []
for text in texts:
    result.append(pos_tag(text))
        
for review in result:
    # 从后向前删除不符合条件的 防止溢出
    for i in range(len(review)-1,-1,-1):
        if review[i][1] not in corps:
            del review[i]

# 词性还原
wnl = WordNetLemmatizer()
texts_lemmated = []    
for review in result:
    temp = []
    if len(review) != 0:
        for word, tag in review:
            if tag.startswith('NN'): # noun
                temp.append(wnl.lemmatize(word, pos='n'))
            elif tag.startswith('VB'): # verb
                temp.append(wnl.lemmatize(word, pos='v'))
            elif tag.startswith('JJ'): # adj
                temp.append(wnl.lemmatize(word, pos='a'))
    texts_lemmated.append(temp)
                    
sentences_processed = [' '.join(text) for text in texts_lemmated]
print(sentences_processed[:3])

[nltk_data] Error loading stopwords: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 11004] getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno 11004]
[nltk_data]     getaddrinfo failed>


['fail zoom app', 'zoom access camera want use app', 'zoom freeze', 'time problem zoom app', 'zoom awesome app stay touch friend family member state something thing screen freeze', 'zoom freeze', 'zoom freeze several time middle meet late version', 'zoom app work bug fix update', 'zoom bad app user use zoom', 'past week take zoom io app crash min', 'apple resolve problem uninstalling zoom server', 'disappoint read experience issue zoom app', 'zoom glitchy app', 'zoom access camera time past bad issue', 'zoom technical difficulty app perfect', 'know app device zoom seem problem enter zoom call screen frozen cannot anything call device', 'zoom try fail', 'zoom let sign app', 'time zoom great freeze', 'select tell others zoom app crash', 'zoom fail share content invoke', 'use bug shut application screen share zoom crash late version', 'disappoint read experience issue zoom client interfering volume apps', 'zoom keep freeze ipad', 'zoom work instal recommended security o update', 'zoom iss

In [16]:
# This step is to calculate similarity with Word2Vec model

rn_vector = avg_feature_vector("iOS screen share support", model=wordvec, num_features=300, index2word_set=index2word_set)

sims2 = []
index = 0

for sentence in sentences_processed:
    vector = avg_feature_vector(sentence, model=wordvec, num_features=300, index2word_set=index2word_set)
    sim = util.pytorch_cos_sim(rn_vector, vector)
    print('process:{0}%'.format(round(index+1)*100/l),end='\r')
    index += 1
    sims2.append(float(sim[0][0])) 

In [17]:
# To dictionary

dic_2 = defaultdict(int)

for i in range(len(sentences)):
    dic_2[reviews[i]] = sims2[i]

In [18]:
# Get top80
dic_2_order=sorted(dic_2.items(),key=lambda x:x[1],reverse=True)
Word2Vec_top80s=dic_2_order[:80]
print(len(Word2Vec_top80s))
print(Word2Vec_top80s[:3])

80
[('5&2021-04-29 19:37:46&Screen sharing ', 0.8209477663040161), ('5&2021-03-30 05:43:10&You d an also screen share ', 0.8209477663040161), ('1&2021-12-28 10:23:59&Could you share your screen ', 0.8209477663040161)]


In [19]:
#Get the Intersection of two models

word2vec = []
sbert = []
for pair in SBert_top80s:
    sbert.append(pair[0])
for pair in Word2Vec_top80s:
    word2vec.append(pair[0])

from collections import Counter

def intersection(a, b):
    c1 = Counter(a)
    temp = []
    for i in b:
        if i in c1 and c1[i]>0:
            temp.append(i)
            c1[i] -= 1
    return temp

res = intersection(sbert,word2vec)
print(res)
    

['5&2021-04-29 19:37:46&Screen sharing ', '1&2020-05-07 02:41:31&the screen sharing doesn t work', '4&2020-04-02 03:42:25&I have been trying to share my screen on my meetings and the admin said that screen sharing was on ', '1&2021-01-07 00:24:03&The screen sharing on these devices are awful because I can t share my screen ', '4&2020-07-15 08:17:02&I like this app especially the share screen ', '2&2021-02-20 11:37:39&You can not screen share on a iPad ', '1&2020-10-02 23:48:09&Can t share my screen on iPad', '2&2020-12-15 04:57:30&When sharing screen only photo sharing option no screen option ', '4&2020-04-16 07:22:42&But I don t like how when someone is screen sharing you can draw on their screen ', '1&2020-05-12 01:52:08&Share screen isn t very good', '1&2020-08-21 00:20:31&Won t let me see other people s screen when they screen share ', '1&2020-05-19 05:21:42&Won t let me see other people s screen when they screen share ', '4&2020-12-16 02:58:49&Also there is no files option for IPa

In [21]:
# write data in txt 

file_handle = open("Intersection\\ZOOM\\ZOOM_01.txt",mode="w")
for r in res:
    file_handle.write(r+"\n")
file_handle.close()